In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Table of Contents</a></span></li><li><span><a href="#Motivation/Problem-Statement" data-toc-modified-id="Motivation/Problem-Statement-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Motivation/Problem Statement</a></span><ul class="toc-item"><li><span><a href="#Example" data-toc-modified-id="Example-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example</a></span></li><li><span><a href="#Code-requirements" data-toc-modified-id="Code-requirements-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Code requirements</a></span></li><li><span><a href="#Exposition-/-other-notebooks" data-toc-modified-id="Exposition-/-other-notebooks-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Exposition / other notebooks</a></span></li></ul></li><li><span><a href="#Representing-and-working-with-probability-distributions" data-toc-modified-id="Representing-and-working-with-probability-distributions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Representing and working with probability distributions</a></span><ul class="toc-item"><li><span><a href="#Examples-illustrating-usage" data-toc-modified-id="Examples-illustrating-usage-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Examples illustrating usage</a></span><ul class="toc-item"><li><span><a href="#Representing-events-as-predicates-and-spaces-as-sets" data-toc-modified-id="Representing-events-as-predicates-and-spaces-as-sets-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Representing events as predicates and spaces as sets</a></span></li><li><span><a href="#Representing-events-as-sets-and-spaces-as-sets" data-toc-modified-id="Representing-events-as-sets-and-spaces-as-sets-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Representing events as sets and spaces as sets</a></span></li><li><span><a href="#Representing-events-as-predicates-and-spaces-as-sets" data-toc-modified-id="Representing-events-as-predicates-and-spaces-as-sets-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Representing events as predicates and spaces as sets</a></span></li><li><span><a href="#Joint-distributions-of-two-independent-variables" data-toc-modified-id="Joint-distributions-of-two-independent-variables-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Joint distributions of two independent variables</a></span></li><li><span><a href="#Defining-distributions-in-terms-of-dictionaries/ProbDists-and-combinations-of-them" data-toc-modified-id="Defining-distributions-in-terms-of-dictionaries/ProbDists-and-combinations-of-them-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Defining distributions in terms of dictionaries/ProbDists and combinations of them</a></span></li><li><span><a href="#Using-the-'conditioning'-operator-&amp;" data-toc-modified-id="Using-the-'conditioning'-operator-&amp;-3.1.6"><span class="toc-item-num">3.1.6&nbsp;&nbsp;</span>Using the 'conditioning' operator &amp;</a></span></li><li><span><a href="#An-alternative-definition-of-joint-distributions-defined-on-arbitrarily-many-independent-distributions" data-toc-modified-id="An-alternative-definition-of-joint-distributions-defined-on-arbitrarily-many-independent-distributions-3.1.7"><span class="toc-item-num">3.1.7&nbsp;&nbsp;</span>An alternative definition of joint distributions defined on arbitrarily many independent distributions</a></span></li></ul></li><li><span><a href="#Code-for-generating-samples-from-distributions" data-toc-modified-id="Code-for-generating-samples-from-distributions-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Code for generating samples from distributions</a></span><ul class="toc-item"><li><span><a href="#Examples-illustrating-usage" data-toc-modified-id="Examples-illustrating-usage-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Examples illustrating usage</a></span></li></ul></li></ul></li></ul></div>

**Notebook author:** emeinhardt@ucsd.edu

# Motivation/Problem Statement

The goal of this collection of notebooks ('Model Notebook k', $k \in \{0, 1, 2, 3\}$) is to document/develop code for calculations related to an isolated word recognition task where:

 - A *lexicon* $\mathcal{L}$ is a set of **wordforms**  $w$, where a **wordform** is a finite sequence $s_0, s_1, s_2 ... s_f$ of segments, and $s_0^i$ denotes the prefix of some wordform $w \in \mathcal{L}$ such that $f = |w| - 1$ and such that $w$ begins with segments $s_0, s_1, ..., s_i$, where $0 \leq i \leq f$.
 - In a single episode of the task, a **speaker** samples a wordform $w = s_0^f$ from $\mathcal{L} \sim p(s_0^f)$.
 - The speaker incrementally produces their intended wordform (one segment at a time, for our purposes) and the **listener** incrementally perceives $\sigma_0^i \sim p(\sigma_0^i | s_0^i)$.
 - The listener considers what they have perceived so far $\sigma_0^i$ and reasons about what the most likely actual intended wordform of the speaker is $\sim p(\hat{s_0^f}|\sigma_0^i) \propto p(\sigma_0^i|s_0^f) p(s_0^f) = p(\sigma_0^i|s_0^i) p(s_0^i)$.


In particular, we want, for each possible intended wordform $s_0^f \in \mathcal{L}$, for each possible prefix $s_0^i$ of $s_0^f$, the listener's expected distribution over the speaker's intended wordform given the actual prefix produced so far $s_0^i$, where the expectation is taken with respect to possible perceived sequences $\sigma_0^i$: 

$$p(\hat{s_0^f}|s_0^i) = \sum_\limits{\sigma_0^i} p(s_0^f|\sigma_0^i)p(\sigma_0^i|s_0^i) \propto \sum_\limits{\sigma_0^i} p(\sigma_0^i|\hat{s_0^f})p(\hat{s_0^f})p(\sigma_0^i|s_0^i)$$

## Example

For example, in any given run of this task for English,

 - The speaker could intend to produce $s_0^f = $\fixme (*cigarette*).
 - The listener could perceive on some occasion (whether due to production, environmental, or perceptual noise) $\sigma_0^f = $ \fixme (*shigarette*).
 - Using their knowledge of the lexicon of English (including an associated prior over wordforms = a joint distribution over segment sequences) and the communication channel (including e.g. that $[s]$ is confusable with $[ʃ]$), they reason about what the most likely intended wordform $\hat{s_0^f}$ is given their percept $\sigma_0^{i=f}$ and conclude that the most likely intended wordform is \FIXME (*cigarette*): they calculate $p(\hat{s_0^i}|\sigma_0^i), \forall i \in 0, 1, 2 ... f$




## Code requirements

The code for calculating 
$$p(\hat{s_0^f}|s_0^i) = \sum_\limits{\sigma_0^i} p(s_0^f|\sigma_0^i)p(\sigma_0^i|s_0^i) \propto \sum_\limits{\sigma_0^i} p(\sigma_0^i|\hat{s_0^f})p(\hat{s_0^f})p(\sigma_0^i|s_0^i)$$
needs to take only two basic inputs:

 - a prior distribution over the lexicon $p(s_0^f) = p(\hat{s_0^f})$.
 - an incrementally defined channel distribution $p(\sigma_0^i|s_0^i) = p(\sigma_0^i|\hat{s_0^i})$, defined (for now) in terms of a uniphone channel distribution $p(\sigma_i|s_i)$, where the distribution over what segment the listener perceives as the $i$th one depends (by assumption) *only* on the actual $i$th segment $s_i$ the speaker produced (if they've actually produced it at the time we're asking about).

from which it needs to generate all relevant code and representations of probability distributions or means of estimating them.

## Exposition / other notebooks

*This* notebook introduces the model being developed and implemented, the organization of other notebooks in the collection, and introduces the adapted form of Peter Norvig's code for conveniently representing and manipulating probability distributions that I make use of in other notebooks.

*Notebook 1* introduces the final model implementation and the derivations underlying its implementation and does so using randomly generated but highly constrained binary lexicons with a simple noise model. It is intended to introduce the model implementation/derivation in a context where the behavior and purpose (and basic correctness) of code is discernable by going through the notebook and the generated example.

*Notebook 2* is more for testing purposes. It contains multiple implementations of the model. One is defined entirely in terms of the abstractions Peter Norvig provides -- abstractions that cannot scale but which were easy to use and whose correctness I have high confidence in. The second implementation is the one presented in notebooks 1 and 3. 

*Notebook 3* is a demonstration notebook where real data is loaded and basic queries are calculated.

# Representing and working with probability distributions

Below is the code used (not exclusively, but often) in subsequent notebooks to represent, manipulate, make queries about, and sample from probability distributions. This section is meant to be a reference/introduction to save space and clutter in subsequent notebooks.

Most of this code is courtesy of P. Norvig, taken from 
 - http://nbviewer.jupyter.org/url/norvig.com/ipython/Probability.ipynb
 - http://nbviewer.jupyter.org/url/norvig.com/ipython/ProbabilityParadox.ipynb

with slight modification. You could plausibly read both of these (mostly the first) to see what structures and operations Norvig defines. The advantage of going through what's below instead is that the data structures and operations are defined once and then worked with, instead of being cumulatively redefined over the course of two notebooks.

In [4]:
import random

from fractions import Fraction

from collections import defaultdict, Counter

is_predicate = callable

def P(event, space): 
    """The probability of an event, given a sample space of equiprobable outcomes. 
    event: a collection of outcomes, or a predicate that is true of outcomes in the event. 
    space: a set of outcomes or a probability distribution of {outcome: frequency} pairs."""
    if is_predicate(event):
        event = such_that(event, space)
    if isinstance(space, ProbDist):
        return sum(space[o] for o in space if o in event)
    else:
#         print(type(event))
#         print(type(space))
        return Fraction(len(event & space), len(space))
    
def such_that(predicate, space): 
    """The outcomes in the sample pace for which the predicate is true.
    If space is a set, return a subset {outcome,...};
    if space is a ProbDist, return a ProbDist {outcome: frequency,...};
    in both cases only with outcomes where predicate(element) is true."""
    if isinstance(space, ProbDist):
        return ProbDist({o:space[o] for o in space if predicate(o)})
    else:
        return {o for o in space if predicate(o)}

# class ProbDist(dict):
class ProbDist(Counter):
    "A Probability Distribution; an {outcome: probability} mapping where probabilities sum to 1."
    def __init__(self, mapping=(), **kwargs):
        self.update(mapping, **kwargs)
        total = sum(self.values())
        if isinstance(total, int): 
            total = Fraction(total, 1)
        for key in self: # Make probabilities sum to 1.
            self[key] = self[key] / total
            
    def __and__(self, predicate): # Call this method by writing `probdist & predicate`
        "A new ProbDist, restricted to the outcomes of this ProbDist for which the predicate is true."
        return ProbDist({e:self[e] for e in self if predicate(e)})
    
    def __repr__(self):
        s = ""
        for k in self:
            if isinstance(self[k], Fraction):
                s+="{0}: {2}/{3} = {1}\n".format(k.__repr__(), float(self[k]), self[k].numerator, self[k].denominator)
            else:
                s+="{0}: {1}\n".format(k.__repr__(), float(self[k]))
        return s

def Uniform(outcomes): return ProbDist({e: 1 for e in outcomes})

def joint(A, B):
    """The joint distribution of two independent probability distributions. 
    Result is all entries of the form {(a, b): P(a) * P(b)}"""
    return ProbDist({(a,b): A[a] * B[b]
                    for a in A
                    for b in B})

## Examples illustrating usage

### Representing events as predicates and spaces as sets

Let $D = \{1,2,3,4,5,6\}$. What proportion of elements in $D$ are $even$?

In [5]:
D    = {1, 2, 3, 4, 5, 6}
even = lambda n: n % 2 == 0
such_that(even, D)
P(even, D)

{2, 4, 6}

Fraction(1, 2)

### Representing events as sets and spaces as sets

What proportion of $D$ is also in $E = \{1, 2, 3\}$

In [7]:
P({1, 2, 3}, D)

Fraction(1, 2)

### Representing events as predicates and spaces as sets

What proportion of the digits from $0-9$ are *even*?

In [9]:
even = lambda n: n % 2 == 0
even(2)
even(3)
set(range(10))
P(even, set(range(10)))

True

False

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

Fraction(1, 2)

### Joint distributions of two independent variables

Suppose we pick one candy from each of two bags of M'n'Ms, one with 
 - 30 brown, 20 yellow, 20 red, 10 green, 10 orange, and 10 tan candies

and the other with
 - 24 blue, 20 green, 16 orange, 14 yellow, 13 red, and 13 brown candies.
 
 What's the distribution over ways of picking first from the first bag and then from the second bag?

In [8]:
bag94 = ProbDist(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
bag96 = ProbDist(blue=24, green=20, orange=16, yellow=14, red=13, brown=13)
MM = joint(bag94, bag96)
MM

ProbDist({('brown', 'blue'): Fraction(9, 125),
          ('brown', 'brown'): Fraction(39, 1000),
          ('brown', 'green'): Fraction(3, 50),
          ('brown', 'orange'): Fraction(6, 125),
          ('brown', 'red'): Fraction(39, 1000),
          ('brown', 'yellow'): Fraction(21, 500),
          ('green', 'blue'): Fraction(3, 125),
          ('green', 'brown'): Fraction(13, 1000),
          ('green', 'green'): Fraction(1, 50),
          ('green', 'orange'): Fraction(2, 125),
          ('green', 'red'): Fraction(13, 1000),
          ('green', 'yellow'): Fraction(7, 500),
          ('orange', 'blue'): Fraction(3, 125),
          ('orange', 'brown'): Fraction(13, 1000),
          ('orange', 'green'): Fraction(1, 50),
          ('orange', 'orange'): Fraction(2, 125),
          ('orange', 'red'): Fraction(13, 1000),
          ('orange', 'yellow'): Fraction(7, 500),
          ('red', 'blue'): Fraction(6, 125),
          ('red', 'brown'): Fraction(13, 500),
          ('red', 'green'): Fra

### Defining distributions in terms of dictionaries/ProbDists and combinations of them

In [10]:
fairCoinDist = ProbDist({"H":0.5, "T":0.5})
print("p(fair coin outcome C)")
fairCoinDist

p(fair coin outcome C)


ProbDist({'H': 0.5, 'T': 0.5})

In [11]:
fairDieDist = Uniform(set(range(1,7)))
print("p(fair die outcome D)")
fairDieDist

p(fair die outcome D)


ProbDist({1: Fraction(1, 6),
          2: Fraction(1, 6),
          3: Fraction(1, 6),
          4: Fraction(1, 6),
          5: Fraction(1, 6),
          6: Fraction(1, 6)})

In [12]:
print("p(C,D)")
coinAndDie = joint(fairCoinDist, fairDieDist)
coinAndDie

p(C,D)


ProbDist({('H', 1): 0.08333333333333333,
          ('H', 2): 0.08333333333333333,
          ('H', 3): 0.08333333333333333,
          ('H', 4): 0.08333333333333333,
          ('H', 5): 0.08333333333333333,
          ('H', 6): 0.08333333333333333,
          ('T', 1): 0.08333333333333333,
          ('T', 2): 0.08333333333333333,
          ('T', 3): 0.08333333333333333,
          ('T', 4): 0.08333333333333333,
          ('T', 5): 0.08333333333333333,
          ('T', 6): 0.08333333333333333})

### Using the 'conditioning' operator &

In [13]:
print("p(C|D = 3)")
dIs3 = lambda cd: cd[1] == 3
coinAndDie & dIs3

p(C|D = 3)


ProbDist({('H', 3): 0.5, ('T', 3): 0.5})

In [14]:
print("p(C=H)")
P(lambda cd: cd[0] == 'H', coinAndDie)

print("p(C=H|D = 3)")
P(lambda cd: cd[0] == 'H', coinAndDie & dIs3)

p(C=H)


0.49999999999999994

p(C=H|D = 3)


0.5

### An alternative definition of joint distributions defined on arbitrarily many independent distributions

In [15]:
from itertools import product
list(product(*[fairCoinDist, fairDieDist]))

[('H', 1),
 ('H', 2),
 ('H', 3),
 ('H', 4),
 ('H', 5),
 ('H', 6),
 ('T', 1),
 ('T', 2),
 ('T', 3),
 ('T', 4),
 ('T', 5),
 ('T', 6)]

In [16]:
from functools import reduce
import operator

def prod(iterable):
    return reduce(operator.mul, iterable, 1)
prod([2,2,2])

8

In [17]:
q = ["a","b","c"]
for i,each in enumerate(q):
    i, each

(0, 'a')

(1, 'b')

(2, 'c')

In [18]:
iter_of_dists = [fairCoinDist, fairDieDist]
iter_of_dists

{each : prod(dist[each[i]] for i,dist in enumerate(iter_of_dists)) for each in list(product(*iter_of_dists))}

[ProbDist({'H': 0.5, 'T': 0.5}),
 ProbDist({1: Fraction(1, 6),
           2: Fraction(1, 6),
           3: Fraction(1, 6),
           4: Fraction(1, 6),
           5: Fraction(1, 6),
           6: Fraction(1, 6)})]

{('H', 1): 0.08333333333333333,
 ('H', 2): 0.08333333333333333,
 ('H', 3): 0.08333333333333333,
 ('H', 4): 0.08333333333333333,
 ('H', 5): 0.08333333333333333,
 ('H', 6): 0.08333333333333333,
 ('T', 1): 0.08333333333333333,
 ('T', 2): 0.08333333333333333,
 ('T', 3): 0.08333333333333333,
 ('T', 4): 0.08333333333333333,
 ('T', 5): 0.08333333333333333,
 ('T', 6): 0.08333333333333333}

In [19]:
def joint2(iter_of_dists):
    #ProbDist({(a,b): A[a] * B[b] for a in A for b in B})
    #ProbDist({ab: A[ab[0]] * B[ab[1]] for ab in product(A,B)})
    return ProbDist({each : prod(dist[each[i]] for i,dist in enumerate(iter_of_dists)) for each in list(product(*iter_of_dists))})
joint2([fairCoinDist, fairDieDist])
joint2([fairCoinDist, fairDieDist, fairCoinDist])

ProbDist({('H', 1): 0.08333333333333333,
          ('H', 2): 0.08333333333333333,
          ('H', 3): 0.08333333333333333,
          ('H', 4): 0.08333333333333333,
          ('H', 5): 0.08333333333333333,
          ('H', 6): 0.08333333333333333,
          ('T', 1): 0.08333333333333333,
          ('T', 2): 0.08333333333333333,
          ('T', 3): 0.08333333333333333,
          ('T', 4): 0.08333333333333333,
          ('T', 5): 0.08333333333333333,
          ('T', 6): 0.08333333333333333})

ProbDist({('H', 1, 'H'): 0.041666666666666685,
          ('H', 1, 'T'): 0.041666666666666685,
          ('H', 2, 'H'): 0.041666666666666685,
          ('H', 2, 'T'): 0.041666666666666685,
          ('H', 3, 'H'): 0.041666666666666685,
          ('H', 3, 'T'): 0.041666666666666685,
          ('H', 4, 'H'): 0.041666666666666685,
          ('H', 4, 'T'): 0.041666666666666685,
          ('H', 5, 'H'): 0.041666666666666685,
          ('H', 5, 'T'): 0.041666666666666685,
          ('H', 6, 'H'): 0.041666666666666685,
          ('H', 6, 'T'): 0.041666666666666685,
          ('T', 1, 'H'): 0.041666666666666685,
          ('T', 1, 'T'): 0.041666666666666685,
          ('T', 2, 'H'): 0.041666666666666685,
          ('T', 2, 'T'): 0.041666666666666685,
          ('T', 3, 'H'): 0.041666666666666685,
          ('T', 3, 'T'): 0.041666666666666685,
          ('T', 4, 'H'): 0.041666666666666685,
          ('T', 4, 'T'): 0.041666666666666685,
          ('T', 5, 'H'): 0.041666666666666685,
          ('T

## Code for generating samples from distributions

In [20]:
#bookkeeping - don't worry about this function...
def getBoundaryVal(dist, i):
    #p = {a:1/2, b:1/2} -> 2 items -> 2-1=1 boundaries where 
    #                   the boundary separating 'a' (item 0) from 'b' (item 1) is at 0+p(a)
    #q = {0:1/3, 1:1/3, 2:1/6, 3:1/6} -> 4 items -> 4-1=3 boundaries where 
    #                   the boundary separating 0 from 1   is at 0 + q(0) = 1/3,
    #                   the boundary separating 1 from 2   is at 0 + q(0) + q(1) = 2/3,
    #                   the boundary separating 2 from 3   is at 0 + q(0) + q(1) + q(2) = 5/6,
    #                ...the boundary separating i from i+1 is at \sum_{j=0}^{j=i} q(j)
    outcomes = list(dist.keys())
    if i >= len(outcomes) - 1:
        raise Exception("Boundary i = {0} out of bounds / does not exist for distribution {1} with {2} outcomes.".format(i, dist, len(outcomes)))
    if i == 0:
        return dist[outcomes[0]]
    return dist[outcomes[i]] + getBoundaryVal(dist, i-1)

#bookkeeping - don't worry about this function...
def getSampleOutcomeIndex(randReal, boundariesLeft, currIndex):
#     print("boundariesLeft: {0}".format(boundariesLeft))
#     print("currIndex: {0}".format(currIndex))
    if boundariesLeft == [] or randReal <= boundariesLeft[0]:
        return currIndex
    return getSampleOutcomeIndex(randReal, boundariesLeft[1:], currIndex + 1)


def sampleFrom(dist, num_samples = None):
    """
    Given a distribution (either an {outcome: probability} mapping where the 
    probabilities sum to 1 or an implicit definition of a distribution via a thunk), 
    this returns a single sample from the distribution, unless num_samples is specified, 
    in which case a generator with num_samples samples is returned.
    """
    if num_samples == None:
        if callable(dist):
            return dist()
        elif isinstance(dist, ProbDist):
            outcomes = list(dist.keys())
        #     print("outcomes: {0}".format(outcomes))

            boundaries = [getBoundaryVal(dist, i) for i in range(len(outcomes)-1)]
        #     print("boundaries: {0}".format(boundaries))

            randVal = random.random() #random real from unit interval
        #     print("randval: {0}".format(randVal))

            sampledOutcomeIndex = getSampleOutcomeIndex(randVal, boundaries, 0)
        #     print("sampledOutcomeIndex: {0}".format(sampledOutcomeIndex))

            sampledOutcome = outcomes[sampledOutcomeIndex]
        #     print("sampledOutcome: {0}".format(sampledOutcome))
            return sampledOutcome
#         else:
#             return dist()
    else:
        if callable(dist):
            return (dist() for each in range(num_samples))
        elif isinstance(dist, ProbDist):
            return (sampleFrom(dist, num_samples = None) for each in range(num_samples))
#         else:
#             return (dist() for each in range(num_samples))

from collections import Counter

def frequencies(samples):
    return Counter(samples)

def makeSampler(dist):
    """
    Given a ProbDist, returns a thunk that when called, returns one sample from dist.
    """
    return lambda: sampleFrom(dist)

### Examples illustrating usage

The main function of interest is 'sampleFrom', which must be passed a thunk (which is presumed to generate samples when called) or a ProbDist (which is then sampled from). By default, 'sampleFrom' returns a single sample; an optional second argument indicates how many samples to collect.

Let's sample from a distribution...

In [21]:
p0 = ProbDist({'a':1/2, 'b':1/2})
p0
sampleFrom(p0)
sampleFrom(p0)

ProbDist({'a': 0.5, 'b': 0.5})

'a'

'b'

...and now let's define a new ProbDist in terms of the observed frequencies from those samples.

In [22]:
samps = list(sampleFrom(p0, 10))
samps
frequencies(samps)
ProbDist(frequencies(samps))

['b', 'b', 'b', 'b', 'b', 'a', 'b', 'a', 'b', 'b']

Counter({'a': 2, 'b': 8})

ProbDist({'a': Fraction(1, 5), 'b': Fraction(4, 5)})

In [26]:
p1 = ProbDist({0:1/3, 1:1/3, 2:1/6, 3:1/6})
p1
sampleFrom(p1)
sampleFrom(p1)

ProbDist({0: 0.33333333333333337,
          1: 0.33333333333333337,
          2: 0.16666666666666669,
          3: 0.16666666666666669})

1

3

In [27]:
samps = list(sampleFrom(p1, 10))
samps
frequencies(samps)
ProbDist(frequencies(samps))

[1, 3, 2, 0, 2, 3, 1, 1, 2, 1]

Counter({0: 1, 1: 4, 2: 3, 3: 2})

ProbDist({0: Fraction(1, 10),
          1: Fraction(2, 5),
          2: Fraction(3, 10),
          3: Fraction(1, 5)})

In [28]:
samps = list(sampleFrom(makeSampler(p1), 10))
samps
frequencies(samps)

[0, 3, 2, 0, 0, 2, 1, 1, 2, 1]

Counter({0: 3, 1: 3, 2: 3, 3: 1})